In [34]:
from cloudvolume import CloudVolume
from meshparty import skeletonize, trimesh_io
from caveclient import CAVEclient
import trimesh
import numpy as np
import datetime
import networkx as nx
from scipy.sparse import identity
from scipy.spatial import distance_matrix
import scipy 
from tqdm import tqdm
# import aws
import pandas as pd
import csv
import pyembree
import matplotlib.pyplot as plt
import scipy.spatial as spatial

### DATAFRAME FOR NEURONS AND ORPHANS

In [35]:
orphans = pd.read_csv("/Users/sheeltanna/Desktop/AGT_REPO/campfire/tip_finding/Orphans.csv")
print(orphans)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/sheeltanna/Desktop/AGT_REPO/campfire/tip_finding/Orphans.csv'

In [ ]:
neurons = pd.read_csv("/Users/sheeltanna/Desktop/AGT_REPO/campfire/tip_finding/Neuron.csv")

In [ ]:
def my_array(x):
    res = list(map(str.strip, x.split('; ')))
    return res

In [ ]:
orphans['endpoints'] = orphans['endpoints'].map(lambda x: list(map(str.strip, x.split('; '))))
print(orphans)

                 seg_id  num_endpoints  \
0    864691135909994000              2   
1    864691135247440303              3   
2    864691134794123793              2   
3    864691135772363453              7   
4    864691135314714227              2   
..                  ...            ...   
127  864691135319600870              2   
128  864691136558839249              2   
129  864691135012763638             -1   
130  864691135458639120              2   
131  864691135826764827              2   

                                             endpoints  
0    [(402584, 228856, 23991), (402985, 229235, 235...  
1    [(401612, 224623, 23991), (405257, 226318, 236...  
2    [(401242, 228382, 24444), (400982, 228457, 245...  
3    [(401289, 218721, 23991), (399895, 216533, 235...  
4    [(397867, 232230, 23999), (397854, 232252, 239...  
..                                                 ...  
127   [(106608, 111803, 20796), (98290, 97555, 21045)]  
128  [(103138, 188530, 21200), (91780, 

In [ ]:
neurons['endpoints'] = neurons['endpoints'].map(lambda x: list(map(str.strip, x.split('; '))))
print(neurons)

               Neurons  num_endpoints  \
0   864691136577830164              6   
1   864691135683615218             13   
2   864691135864989916              2   
3   864691135837578899              9   
4   864691135501859650             14   
5   864691135544674088              7   
6   864691135614368971             12   
7   864691135334773481              7   
8   864691135614361291             12   
9   864691135445907602             12   
10  864691136390697343              8   
11  864691135396741921              9   
12  864691135360418631              6   
13  864691135463797061              6   
14  864691135526309723              9   

                                            endpoints  
0   [(402188, 228684, 24029), (401258, 224832, 240...  
1   [(77233, 113188, 20454), (100382, 143836, 2164...  
2   [(211973, 200313, 23233), (167751, 211623, 207...  
3   [(351264, 143646, 15182), (344801, 142291, 169...  
4   [(101840, 247377, 19543), (92823, 255328, 2038...  
5   [(2

In [ ]:
#check
for index, row in orphans.iterrows():
     for endpoint in row["endpoints"]:
         print(endpoint)

(402584, 228856, 23991)
(402985, 229235, 23554)
(401612, 224623, 23991)
(405257, 226318, 23620)
(409162, 219642, 23891)
(401242, 228382, 24444)
(400982, 228457, 24513)
(401289, 218721, 23991)
(399895, 216533, 23526)
(406366, 215719, 23621)
(403940, 210639, 23991)
(394569, 213255, 23991)
(376682, 175983, 22020)
(365111, 191189, 24020)
(371210, 191781, 21770)
(397867, 232230, 23999)
(397854, 232252, 23997)
(403332, 227545, 24398)
(403566, 227614, 24425)
(77262, 113088, 20442),
(79178, 109116, 20402)
(100475, 143982, 21654)
(100583, 144194, 21702)
(80890, 147158, 20708)
(81592, 148549, 20453)
(81373, 148139, 20453)
(83362, 154016, 20454)
(83471, 163193, 20640)
(71898, 146194, 20442)
(71499, 146291, 20404)
(78610, 145071, 20707)
(75884, 143590, 20453)
(78226, 151636, 19714)
(66549, 151567, 19790)
(78090, 150130, 20388)
(77575, 107807, 21133)
(77000, 99565, 21215)
(79785, 135709, 21133)
(84176, 127514, 21582)
(80523, 134733, 21498)
(81198, 106660, 21133)
(88864, 99498, 20482)
(77826, 139204

### FUNCTIONS FOR TIP FINDER

In [ ]:
def get_and_process_mesh(root_id):
    datastack_name = "minnie65_phase3_v1"
    client = CAVEclient(datastack_name)
    vol = CloudVolume(
        client.info.segmentation_source(),
        use_https=True,
        progress=False,
        bounded=False,
        fill_missing=True,
        secrets={"token": client.auth.token}
    )
    print("Downloading Mesh")
    mesh = vol.mesh.get(str(root_id))[root_id]
    mesh_obj = trimesh.Trimesh(np.divide(mesh.vertices, np.array([1,1,1])), mesh.faces)
    print("Vertices: ", mesh.vertices.shape[0])

    if mesh_obj.volume > 4000000000000:
        print("TOO BIG, SKIPPING")
        #queue_url_endpoints = sqs.get_or_create_queue("root_ids_functional_dlqueue")

        #entries=sqs.construct_rootid_entries([root_id])

        #sqs.send_batch(queue_url_endpoints, entries)

        return None
    trimesh.repair.fix_normals(mesh_obj)
    mesh_obj.fill_holes()

    return mesh_obj

In [ ]:
def get_soma(soma_id:str):
    cave_client = CAVEclient('minnie65_phase3_v1')
    soma = cave_client.materialize.query_table(
        "nucleus_neuron_svm",
        filter_equal_dict={'id':soma_id}
    )
    return soma

In [ ]:
def process_mesh_ccs(mesh_obj):
    print("Processing CC's")
    ccs_graph = trimesh.graph.connected_components(mesh_obj.edges)
    ccs_len = [len(c) for c in ccs_graph]

    # Subselect the parts of the mesh that are not inside one another 
    # the other components are an artifact of the soma seg and small unfilled sections
    largest_component = ccs_graph[np.argmax(ccs_len)]
    largest_component_remap = np.arange(ccs_graph[np.argmax(ccs_len)].shape[0])
    face_dict = {largest_component[i]:largest_component_remap[i] for i in range(largest_component.shape[0])}

    new_faces_mask = np.isin(mesh_obj.faces, list(face_dict.keys()))
    new_faces_mask = new_faces_mask[:, 0]*new_faces_mask[:, 1]*new_faces_mask[:, 2]

    new_faces = np.vectorize(face_dict.get)(mesh_obj.faces[new_faces_mask])
    new_faces = new_faces[new_faces[:, 0] != None]
    largest_component_mesh = trimesh.Trimesh(mesh_obj.vertices[largest_component], new_faces)

    all_ids = set(largest_component)
    encapsulated_ids = []

    for i in range(1, len(ccs_graph)):
        n_con = largest_component_mesh.contains(mesh_obj.vertices[ccs_graph[i]])
        if np.sum(n_con) / n_con.shape[0] == 0 and n_con.shape[0] > 50:
            all_ids.update(ccs_graph[i])
        else:
            if len(ccs_graph[i]) < 1000:
                encapsulated_ids.append((np.mean(mesh_obj.vertices[ccs_graph[i]], axis=0)/[4,4,40], len(ccs_graph[i])))
            
    all_component = np.array(list(ccs_graph[np.argmax(ccs_len)]))
    all_component_remap = np.arange(all_component.shape[0])
    face_dict = {all_component[i]:all_component_remap[i] for i in range(all_component.shape[0])}
    new_faces_mask = np.isin(mesh_obj.faces, list(face_dict.keys()))
    new_faces_mask = new_faces_mask[:, 0]*new_faces_mask[:, 1]*new_faces_mask[:, 2]

    new_faces = np.vectorize(face_dict.get)(mesh_obj.faces[new_faces_mask])
    new_faces[new_faces[:, 0] != None]
    
    largest_component_mesh = trimesh.Trimesh(mesh_obj.vertices[all_component], new_faces)

    mesh_obj = largest_component_mesh
    return mesh_obj, encapsulated_ids, np.max(ccs_len)

In [ ]:
def process_defects(mesh_obj, a=.75):
    bad_edges = trimesh.grouping.group_rows(
        mesh_obj.edges_sorted, require_count=1)
    bad_edges_ind = mesh_obj.edges[bad_edges]
    sparse_edges = mesh_obj.edges_sparse
    xs = list(bad_edges_ind[:, 0]) + list(bad_edges_ind[:, 1]) 
    ys = list(bad_edges_ind[:, 1]) + list(bad_edges_ind[:, 0])
    vs = [1]*bad_edges_ind.shape[0]*2
    bad_inds = scipy.sparse.coo_matrix((vs, (xs, ys)), shape=(mesh_obj.vertices.shape[0], mesh_obj.vertices.shape[0]))
    # Make it symmetrical and add identity so each integrates from itself too, then subtract singleton edges
    # I noticed that the number of asymmetrical edges vs the number of single edges I find from group rows
    # Are close but different. Haven't looked into that yet. Also removing edges 1 hop away from single edges to remove bias towards
    # Holes in the mesh that are caused by mesh construction errors as opposed to segmentation errors
    sparse_edges = mesh_obj.edges_sparse + mesh_obj.edges_sparse.T + identity(mesh_obj.edges_sparse.shape[0]) - sparse_edges.multiply(bad_inds) - bad_inds
    degs = mesh_obj.vertex_degree + 1

    # N_iter is a smoothing parameter here. The loop below smooths the vertex error about the mesh to get more consistent connected regions
    n_iter = 2
    angle_sum = np.array(abs(mesh_obj.face_angles_sparse).sum(axis=1)).flatten()
    defs = (2 * np.pi) - angle_sum

    abs_defs = np.abs(defs)
    abs_defs_i = abs_defs.copy()
    for i in range(n_iter):
        abs_defs_i = sparse_edges.dot(abs_defs_i) / degs
    
    verts_select = np.argwhere((abs_defs_i > a))# & (abs_defs < 2.5))

    edges_mask = np.isin(mesh_obj.edges, verts_select)
    edges_mask[bad_edges] = False
    edges_select = edges_mask[:, 0] * edges_mask[:, 1]
    edges_select = mesh_obj.edges[edges_select]

    G = nx.from_edgelist(edges_select)#f_edge_sub)

    ccs = nx.connected_components(G)
    subgraphs = [G.subgraph(cc).copy() for cc in ccs]

    lens = []
    lengths = []
    for i in tqdm(range(len(subgraphs))):
        ns = np.array(list(subgraphs[i].nodes()))
    #     ns = ns[abs_defs[ns ]]
        l = len(ns)
        if l > 20 and l < 5000:
            lens.append(ns)
            lengths.append(l)
    all_nodes = set()
    for l in lens:
        all_nodes.update(l)
    all_nodes = np.array(list(all_nodes))
    # sharp_pts = mesh_obj.vertices[all_nodes]
    centers = np.array([np.mean(mesh_obj.vertices[list(ppts)],axis=0) for ppts in lens])

    return centers, lens


In [ ]:
def process_endpoints(mesh_obj, skel_mp):
    # Process the skeleton to get the endpoints
    interior_cc_mask = set()
    el = nx.from_edgelist(skel_mp.edges)
    comps = list(nx.connected_components(el))
    for c in comps:
        if len(c) < 10000:
            n_con = mesh_obj.contains(skel_mp.vertices[list(c)])
            if np.sum(n_con) / n_con.shape[0] > .05:
                interior_cc_mask.update(list(c))
    # Process the skeleton to get the endpoints
    edges = skel_mp.edges.copy()

    edge_mask = ~np.isin(edges, interior_cc_mask)
    edge_mask = edge_mask[:, 0] + edge_mask[:, 1]
    edges = edges[edge_mask]
    edges_flat  = edges.flatten()
    edge_bins = np.bincount(edges_flat) 

    eps = np.squeeze(np.argwhere(edge_bins==1))
    eps_nm = skel_mp.vertices[eps]

    eps_comp = distance_matrix(eps_nm, eps_nm)
    eps_comp[eps_comp == 0] = np.inf
    eps_thresh = np.argwhere(~(np.min(eps_comp, axis=0) < 3000))

    eps = np.squeeze(eps[eps_thresh])
    eps_nm = np.squeeze(eps_nm[eps_thresh])
    return eps, eps_nm

In [ ]:
def process_mesh_errors(mesh_obj, centers, eps, eps_nm, lens, skel_mp):
    print("Processing mesh errors")
    path_to_root_dict = {}
    for ep in eps:
        path_to_root_dict[ep] = skel_mp.path_to_root(ep)
        
    dists_defects = np.zeros(centers.shape[0])
    sizes = np.zeros(centers.shape[0])
    mesh_map = skel_mp.mesh_to_skel_map
    closest_skel_pts = mesh_map[[l[0] for l in lens]]

    # print(centers, eps_nm)

    dist_matrix = distance_matrix(centers, eps_nm)
    ct = 0

    closest_tip = np.zeros((centers.shape[0]))

    for center in tqdm(centers):
    #     skel_pts_dists = np.linalg.norm(skel_mp.vertices - center, axis=1)
    #     ep_pts_dists = np.linalg.norm(eps_nm - center, axis=1)
        
        closest_skel_pt = closest_skel_pts[ct]
        min_ep = np.inf
        eps_hit = []
        for j, ep in enumerate(eps):
            if closest_skel_pt in path_to_root_dict[ep]:
                eps_hit.append(j)
        if len(eps_hit) == 0:
            dists_defects[ct] = np.inf
            sizes[ct] = np.inf
            ct+=1
            continue
        
        dists = dist_matrix[ct, eps_hit]
    #     print(dists, eps_hit, center / [4,4,40])
        
        amin = np.argmin(dists)
        tip_hit = eps_hit[amin]
        min_dist = dists[amin]
        
        closest_tip[ct] = tip_hit
    #     print(np.argmin(ep_pts_dists), ep_found, eps_nm[np.argmin(ep_pts_dists)]/[4,4,40], eps_nm[j]/[4,4,40], center/[4,4,40])
        dists_defects[ct] = min_dist
        sizes[ct] = len(lens[ct])
        ct+=1
    dists_defects_sub = dists_defects[dists_defects < np.inf]
    sizes_sub = sizes[dists_defects < np.inf]
    centers_sub = centers[dists_defects < np.inf]
    tips_hit_sub = closest_tip[dists_defects < np.inf]
    closest_skel_pts_sub = closest_skel_pts[dists_defects < np.inf]
    inds_sub = np.arange(centers.shape[0])[dists_defects < np.inf]


    # Also ranking each component based on its PCA- if the first component is big enough, the points are mostly linear
    # These point sets seem to be less likely to be true errors
    from sklearn.decomposition import PCA
    pca_vec = np.zeros(inds_sub.shape[0])
    for i in range(inds_sub.shape[0]):
        pca = PCA()#n_components=2)
        pca.fit(mesh_obj.vertices[lens[inds_sub[i]]])

        pca_vec[i] = pca.explained_variance_ratio_[0]

    dists_defects_sub[dists_defects_sub < 4000] = 100
    dists_defects_norm = dists_defects_sub #/ np.max(dists_defects_sub)
    ranks_ep = sizes_sub / dists_defects_norm * (1-pca_vec)
    ranks = sizes_sub**2 * (1-pca_vec)

    #ranks_ep_errors_filt = ranks_ep[ranks_ep > .1]
    centers_ep_send_errors = centers_sub[np.argsort(ranks_ep)][::-1][:20]
    final_mask_eps = np.full(centers_ep_send_errors.shape[0], True)
    tips_hit_send_ep = tips_hit_sub[np.argsort(ranks_ep)][::-1][:20]
    uns, nums = np.unique(tips_hit_send_ep, return_counts=True)

    for un, num in zip(uns, nums):
        if num > 1:
            final_mask_eps[np.argwhere(tips_hit_send_ep == un)[1:]] = False
    centers_errors_ep = centers_ep_send_errors[final_mask_eps]
    centers_errors = centers_sub[np.argsort(ranks)[::-1]][:20]
    return centers_errors, centers_errors_ep, ranks, ranks_ep, path_to_root_dict




In [ ]:
def process_mesh_facets(mesh_obj, skel_mp, eps, path_to_root_dict, eps_nm):
    print("Processing facets")
    locs = np.argwhere(mesh_obj.facets_area > 50000)

    mesh_map = skel_mp.mesh_to_skel_map
    mesh_coords = mesh_obj.vertices[mesh_obj.faces]
    mean_locs = []
    mesh_ind = []
    fs = []
    for l in tqdm(locs):
        fs.append(np.sum(mesh_obj.facets_area[l]))
        fc = mesh_obj.facets[l[0]]
        vert_locs = mesh_coords[fc]
        mean_locs.append(np.mean(vert_locs[:, 0], axis=0))
        mesh_ind.append(fc[0])
    mesh_ind = mesh_obj.faces[mesh_ind][:, 0]
    mean_locs = np.array(mean_locs)
    dists_defects_facets = np.zeros(mean_locs.shape[0])
    mesh_map_facets = skel_mp.mesh_to_skel_map
    closest_skel_pts_facets = mesh_map[[m for m in mesh_ind]]
    dist_matrix_facets = distance_matrix(mean_locs, eps_nm)
    ct = 0

    closest_tip_facets = np.zeros((mean_locs.shape[0]))

    for center in tqdm(mean_locs):

        closest_skel_pt = closest_skel_pts_facets[ct]
        eps_hit = []
        for j, ep in enumerate(eps):
            if closest_skel_pt in path_to_root_dict[ep]:
                eps_hit.append(j)
        if len(eps_hit) == 0:
            dists_defects_facets[ct] = np.inf
            ct+=1
            continue
        
        dists = dist_matrix_facets[ct, eps_hit]
        
        amin = np.argmin(dists)
        tip_hit = eps_hit[amin]
        min_dist = dists[amin]
        
        closest_tip_facets[ct] = tip_hit
        dists_defects_facets[ct] = min_dist
        ct+=1
    dists_defects_sub_facets = dists_defects_facets[dists_defects_facets < np.inf]
    sizes_sub_facets = np.array(fs)[dists_defects_facets < np.inf]
    mean_locs_facets = mean_locs[dists_defects_facets < np.inf]
    tips_hit_sub_facets = closest_tip_facets[dists_defects_facets < np.inf]
    closest_skel_pts_sub_facets = closest_skel_pts_facets[dists_defects_facets < np.inf]
    inds_sub_facets = np.arange(mean_locs.shape[0])[dists_defects_facets < np.inf]
    ranks_ep_facets = sizes_sub_facets**2 / dists_defects_sub_facets
    #ranks_ep_facets_filt = ranks_ep_facets[ranks_ep_facets > 2e7]
    mean_locs_send_facets = mean_locs_facets[np.argsort(ranks_ep_facets)][::-1][:20]
    final_mask_facets = np.full(mean_locs_send_facets.shape[0], True)
    tips_hit_send_facets = tips_hit_sub_facets[np.argsort(ranks_ep_facets)][::-1][:20]
    uns, nums = np.unique(tips_hit_send_facets, return_counts=True)

    for un, num in zip(uns, nums):
        if num > 1:
            final_mask_facets[np.argwhere(tips_hit_send_facets == un)[1:]] = False
    facets_send_final = mean_locs_send_facets[final_mask_facets] / [4,4,40]
    return facets_send_final, ranks_ep_facets

In [ ]:
#TIP FINDING FUNCTION
def error_locs_defects(root_id, soma_id = None, soma_table=None, center_collapse=True):
    #print("START", root_id)

    mesh_obj = get_and_process_mesh(root_id)
    if mesh_obj is None:
        return None
    # SKELETONIZE - if we are just looking for general errors, not errors at endpoints, this can be skipped
    try:
        if soma_table==None:
            soma_table = get_soma(str(soma_id))
        if soma_table[soma_table.id == soma_id].shape[0] > 0:
            center = np.array(soma_table[soma_table.id == soma_id].pt_position)[0] * [4,4,40]
        else:
            center=None
    except:
        center = None
    print("Subselecting largest connected component of mesh")
    mesh_obj, encapsulated_ids, max_verts = process_mesh_ccs(mesh_obj)

    skel_mp = skeletonize.skeletonize_mesh(trimesh_io.Mesh(mesh_obj.vertices, 
                                            mesh_obj.faces),
                                            invalidation_d=15000,
                                            shape_function='cone',
                                            collapse_function='branch',
#                                             soma_radius = soma_radius,
                                            soma_pt=center,
                                            smooth_neighborhood=5,
                                            cc_vertex_thresh=max_verts - 10
#                                                     collapse_params = {'dynamic_threshold':True}
                                            )
    print("Skel done")

    # find edges that only occur once..  might be faster to find these in the sparse matrix..
    centers, lens = process_defects(mesh_obj)
    eps, eps_nm = process_endpoints(mesh_obj, skel_mp)

    if len(centers) !=0:
        centers_errors, centers_errors_ep, ranks, ranks_ep, path_to_root_dict = process_mesh_errors(mesh_obj, centers, eps, eps_nm, lens, skel_mp)
        ranks_return = np.squeeze(ranks[np.argsort(ranks)[::-1]][:20])
        ranks_ep_return = np.squeeze(ranks_ep[np.argsort(ranks_ep)][::-1][:20])
    else:
        # Assign placeholder values for each of the variables above.
        centers_errors = np.zeros ((1,3))
        centers_errors_ep = np.zeros ((1,3))
        ranks = np.zeros ((1))
        ranks_ep = np.zeros((1, 3))
        path_to_root_dict = {}
        for ep in eps:
            path_to_root_dict[ep] = skel_mp.path_to_root(ep)

        ranks_return = 0
        ranks_ep_return = 0


    #if len(centers_errors.shape) > 1 and centers_errors.shape[0] > 0 and len(centers_errors_ep.shape) > 1 and len(centers_errors_ep.shape[0] > 0):
    #    centers_errors = centers_errors[np.min(distance_matrix(centers_errors, centers_errors_ep), axis=1)>1000]
    facets_send_final, ranks_ep_facets = process_mesh_facets(mesh_obj, skel_mp, eps, path_to_root_dict, eps_nm)

    errors_send = centers_errors / [4,4,40]
    errors_tips_send = centers_errors_ep / [4,4,40]
    encapsulated_centers = [e[0] for e in encapsulated_ids]
    encapsulated_lens = [e[1] for e in encapsulated_ids]
    sorted_encapsulated_send = np.array(encapsulated_centers)[np.argsort(encapsulated_lens)][::-1]



    return sorted_encapsulated_send, facets_send_final, errors_send, errors_tips_send, np.squeeze(ranks_ep_facets[[np.argsort(ranks_ep_facets)][::-1][:20]]), ranks_return, ranks_ep_return



ASSUMING output from error_locs IS: 
facets_find_final: [[x,y,z], [x2,y2,z2], etc]
errors_tips_send: [[x,y,z], [x2,y2,z2], etc]


In [ ]:
points = [[1,2,3], [4,5,6]]
for x in points:
    print(x)

[1, 2, 3]
[4, 5, 6]


In [ ]:
points.append([7,8,9])
print(points)

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]


In [ ]:
array = [[0,0,0]]
arr2 = [3,4,5]
array.append(arr2)
print(array)

[[0, 0, 0], [3, 4, 5]]


In [ ]:
d = {'col1': [1, 2], 'col2': [3, 4]}
df = pd.DataFrame(data=d)
print(df)
df["empty"] = df["col1"].map(lambda x: [[0,0,0]])
print(df)

   col1  col2
0     1     3
1     2     4
   col1  col2        empty
0     1     3  [[0, 0, 0]]
1     2     4  [[0, 0, 0]]


In [ ]:
def generate_endpoints(dataframe) :
    #assume I get in a dataframe with orphan(seg_ID), num endpoints, labelled endpoints
    
    #add collumn to df for predicted endpoints, add an empty array here
    #dataframe["endpoints_generated"] = [[0,0,0]]
    dataframe["endpoints_generated"] = dataframe["seg_id"].map(lambda x: [()])
    #first we need to iterate through the given dataframe row by row
    count = 0
    for index, row in dataframe.iterrows(): 
        #get the neuron/orphan seg_ID
        if count == 3:
            break
        count = count + 1
        seg_id = row["seg_id"]
        #pass the seg_id into the generating function
        sorted_encapsulated_send, facets_send_final, errors_send, errors_tips_send, dummy, dummy2, dummy3 = error_locs_defects(seg_id)
        #add the predicted endpoints into the dataframe row
        for endpoint in facets_send_final:
            #if the endpoint is 0,0,0 then we skip 
            if(np.all(0)):
                break
            row["endpoints_generated"].append(endpoint)
        for endpoints in errors_tips_send:
            if(np.all(0)):
                break
            row["endpoints_generated"].append(endpoint)

In [ ]:
generate_endpoints(orphans)

Vertices:  7993
Subselecting largest connected component of mesh
Processing CC's


100%|██████████| 7992/7992 [00:00<00:00, 1281771.09it/s]


Skel done


100%|██████████| 26/26 [00:00<00:00, 172277.89it/s]


Processing facets


...

100%|██████████| 11/11 [00:00<00:00, 113359.57it/s]


Processing mesh errors


100%|██████████| 1/1 [00:00<00:00, 25575.02it/s]


Processing facets


100%|██████████| 8/8 [00:00<00:00, 96420.78it/s]


In [ ]:
print(orphans["endpoints_generated"])

0      [(), [402970.1091549296, 229214.63028169013, 2...
1      [(), [409125.1111111111, 219484.22222222222, 2...
2                                                   [()]
3                                                   [()]
4                                                   [()]
                             ...                        
127                                                 [()]
128                                                 [()]
129                                                 [()]
130                                                 [()]
131                                                 [()]
Name: endpoints_generated, Length: 132, dtype: object


In [ ]:
for orphan in orphans["endpoints"][0]:
    print(orphan)


(402584, 228856, 23991)
(402985, 229235, 23554)


In [ ]:
#example to see outputs
sorted_encapsulated_send, facets_send_final, errors_send, errors_tips_send, dummy, dummy2, dummy3 = error_locs_defects(864691135909994000)
print(facets_send_final)
print(errors_tips_send)

Vertices:  7993
Subselecting largest connected component of mesh
Processing CC's


100%|██████████| 7992/7992 [00:00<00:00, 1294442.29it/s]


Skel done


100%|██████████| 26/26 [00:00<00:00, 165732.38it/s]


Processing facets


100%|██████████| 6/6 [00:00<00:00, 32640.50it/s]

[[402970.10915493 229214.63028169  23552.55      ]
 [402527.5625     228866.3125      23991.45      ]]
[[0. 0. 0.]]


In [ ]:
# Debugging

x = np.zeros((10,3))
y = np.ones((10,3))
dist_matrix = distance_matrix(x,y)



### ACCURACY FUNCTION

In [ ]:
def pred_eps_acc(gt_endpoints, pred_endpoints, threshold):
    # Calculate distances
    dist_matrix = np.array(spatial.distance.cdist(gt_endpoints, pred_endpoints, metric = 'euclidean'))

    # Apply threshold
    dist_matrix[dist_matrix > threshold] = 0

    # Calculating accuracy
    valid_eps = np.count_nonzero(dist_matrix, axis = 1)
    accuracy = np.count_nonzero(valid_eps) / len(gt_endpoints)

    # If more than one valid endpoint found for a single ground truth endpoint, add the other valid endpoints to extra_valid_pairs
    extra_valid_pairs = []
    [[extra_valid_pairs.append([gt_endpoints[i], pred_endpoints[index]]) \
        for index, j in enumerate(dist_matrix[i]) if j != np.min(dist_matrix[i][dist_matrix[i] != 0]) if j != 0] \
            for i in valid_eps if i > 1]

    return accuracy, extra_valid_pairs

In [ ]:
data = pd.DataFrame(columns=["seg_id", "num_gt_eps", "gt_eps", "num_pred_eps", "pred_eps"])
data.loc[len(data.index)] = [864691136577830164, 6, [
    [402188, 228684, 24029], [401258, 224832, 24029], [401314, 228366, 24424], \
    [400199, 220721, 24029], [397870, 232292, 24004], [403272, 227529, 24394]], 6, \
    [[400292, 220879, 24027], [401253, 228342, 24408], [401253, 228342, 24409],
        [402923, 227458, 24372], [402099, 228716, 24037], [402627, 231471, 24026]]]
data

In [ ]:
acc, extra_valid_pairs = pred_eps_acc(
    data.loc[0, "gt_eps"], data.loc[0, "pred_eps"], 100)
print(acc)
print(extra_valid_pairs)